In [1]:
import pandas as pd

import requests
from bs4 import BeautifulSoup, SoupStrainer
import cchardet

from tqdm import tqdm
import logging
import sys
sys.path.append('../utils')
from utils import get_logger, Clock

In [2]:
logger = logging.getLogger(__name__)
logger = get_logger(logger=logger)

In [3]:
session = requests.Session()
subtle_class = SoupStrainer(class_="subtle")
gen8ou_ladder_page = session.get('https://pokemonshowdown.com/ladder/gen8ou')
soup = BeautifulSoup(gen8ou_ladder_page.text, 'lxml', parse_only=subtle_class)

logger.info("Buscando as tags que possuem os nomes dos top 500 jogadores da ladder...")
top500_list = soup.find_all(class_='subtle')
logger.info("Tags coletadas.")

2022-09-28 23:47:20,271 - INFO - Buscando as tags que possuem os nomes dos top 500 jogadores da ladder...
2022-09-28 23:47:20,275 - INFO - Tags coletadas.


In [4]:
users = []
size = len(top500_list)

logger.info("A iteração começará agora...")
iteration_time = Clock("Construção da lista de usuários")
with tqdm(total=size) as progress_bar:
    progress_bar.set_description("Montando lista com os nomes de usuário")
    for tag in top500_list:
        users.append(tag.contents[0])
        progress_bar.update(1)
iteration_time.stop_watch()

2022-09-28 23:47:20,295 - INFO - A iteração começará agora...
Montando lista com os nomes de usuário: 100%|████████████████████████████████████| 500/500 [00:00<00:00, 495078.38it/s]
2022-09-28 23:47:20,323 - INFO - Construção da lista de usuários levou 00s para ser executado.


Vamos dar uma olhada nos 5 primeiros nomes dos jogadores:

In [5]:
users[:5]

['Barracades', 'Relic Stone', 'pyukumuku guru', 'Chokst du', 'jaJAJJ']

Para a pesquisa de replays por usuário não faz diferença se o nome está minúsculo, maiúsculo, separado por espaço ou não. Portanto, para facilitar o processo, vamos eliminar os espaços:

In [6]:
users = [nick.replace(' ', '') for nick in users]

In [7]:
users[:5]

['Barracades', 'RelicStone', 'pyukumukuguru', 'Chokstdu', 'jaJAJJ']

In [8]:
replay_search_by_user_url = 'https://replay.pokemonshowdown.com/search/?output=html&user='

In [9]:
replay_links_by_user = []
size = len(users)

logger.info("A iteração começará agora...")
iteration_time = Clock("Coleta dos links de replays")
session = requests.Session()
a_tag = SoupStrainer('a')
with tqdm(total=size) as progress_bar:
    progress_bar.set_description("Minerando os links de replay de cada jogador")
    for user in users:
        replay_search_page = session.get(replay_search_by_user_url + user)
        soup = BeautifulSoup(replay_search_page.text, 'lxml', parse_only=a_tag)
        user_replays = [link.get('href') for link in soup.find_all('a')]
        replay_links_by_user.append(user_replays)
        progress_bar.update(1)
iteration_time.stop_watch()

2022-09-28 23:47:20,404 - INFO - A iteração começará agora...
Minerando os links de replay de cada jogador: 100%|██████████████████████████████████| 500/500 [04:16<00:00,  1.95it/s]
2022-09-28 23:51:36,461 - INFO - Coleta dos links de replays levou 04m16s para ser executado.


In [10]:
gen8ou_replay_ids = []
size = len(replay_links_by_user)

logger.info("A iteração começará agora...")
iteration_time = Clock("Filtragem dos links de replays")
with tqdm(total=size) as progress_bar:
    progress_bar.set_description("Filtrando apenas os replays da tier OU (gen8)")
    for user_links in replay_links_by_user:
        for replay_id in user_links:
            if replay_id.startswith('/gen8ou-'):
                gen8ou_replay_ids.append(replay_id)
        progress_bar.update(1)
iteration_time.stop_watch()
logger.info(f"Existem {len(gen8ou_replay_ids)} id's de replay na base!")

2022-09-28 23:51:36,481 - INFO - A iteração começará agora...
Filtrando apenas os replays da tier OU (gen8): 100%|██████████████████████████████| 500/500 [00:00<00:00, 83896.15it/s]
2022-09-28 23:51:36,492 - INFO - Filtragem dos links de replays levou 00s para ser executado.
2022-09-28 23:51:36,494 - INFO - Existem 8249 id's de replay na base!


In [11]:
replay_url = 'https://replay.pokemonshowdown.com/'

In [12]:
teams = []
size = len(gen8ou_replay_ids)

logger.info("A iteração começará agora...")
iteration_time = Clock("Coleta de todas as equipes")
session = requests.Session()
class_log = SoupStrainer(class_="log")
with tqdm(total=size) as progress_bar:
    progress_bar.set_description("Coletando todas as equipes de cada replay")
    for replay_id in gen8ou_replay_ids:
        replay_page = session.get(replay_url + replay_id)
        soup = BeautifulSoup(replay_page.text, 'lxml', parse_only=class_log)
        if soup.find(class_='log'):
            content = soup.find(class_='log').contents[0]
            team = content.split('|poke|')[1:13]
            if(len(team) != 0):
                team[-1] = team[-1].split('\n')[0]
                team = [s.split('|') for s in team]
                team = [s[1] for s in team]
                team = [s.split(',') for s in team]
                team = [s[0] for s in team]
                teams.append(team[:6])
                teams.append(team[6:])
        progress_bar.update(1)
iteration_time.stop_watch()
logger.info(f'Existem {len(teams)} equipes na base!')

2022-09-28 23:51:36,536 - INFO - A iteração começará agora...
Coletando todas as equipes de cada replay: 100%|█████████████████████████████████| 8249/8249 [1:03:31<00:00,  2.16it/s]
2022-09-29 00:55:07,663 - INFO - Coleta de todas as equipes levou 01h03m31s para ser executado.
2022-09-29 00:55:07,663 - INFO - Existem 16494 equipes na base!


In [13]:
columns = ['pokemon_1', 'pokemon_2', 'pokemon_3', 'pokemon_4', 'pokemon_5', 'pokemon_6']
ladder_teams_df = pd.DataFrame(columns=columns)
size = len(teams)

logger.info("A iteração começará agora...")
iteration_time = Clock("Construção do DataFrame")
with tqdm(total=size) as progress_bar:
    progress_bar.set_description("Filtrando apenas as equipes de 6 pokémons e adicionando ao DataFrame")
    for index, team in enumerate(teams):
        if len(team) == 6:
            ladder_teams_df.loc[index] = team
        progress_bar.update(1)
iteration_time.stop_watch()
logger.info(f'Existem {len(ladder_teams_df)} equipes de 6 pokémons na base!')

2022-09-29 00:55:07,703 - INFO - A iteração começará agora...
Filtrando apenas as equipes de 6 pokémons e adicionando ao DataFrame: 100%|█████| 16494/16494 [00:33<00:00, 494.75it/s]
2022-09-29 00:55:41,043 - INFO - Construção do DataFrame levou 33s para ser executado.
2022-09-29 00:55:41,059 - INFO - Existem 16447 equipes de 6 pokémons na base!


In [14]:
ladder_teams_df.head()

,pokemon_1,pokemon_2,pokemon_3,pokemon_4,pokemon_5,pokemon_6
0,Kartana,Urshifu-*,Landorus-Therian,Melmetal,Weavile,Dragonite
1,Mew,Blaziken,Scizor,Garchomp,Ninetales-Alola,Dragapult
2,Garchomp,Tapu Koko,Moltres-Galar,Ferrothorn,Volcarona,Dragonite
3,Landorus-Therian,Weavile,Melmetal,Volcanion,Crawdaunt,Dragapult
4,Garchomp,Aegislash,Urshifu-*,Dragapult,Rillaboom,Bisharp


In [15]:
file_path = 'C:\\Users\\User\\Desktop\\8º período\\MSI I\\Repositórios\\itemset_mining_applied_to_pokemon_teams\\'

ladder_teams_df.to_csv(file_path + 'data\\processed\\ladder_teams.csv', index=False)
ladder_teams_df.to_excel(file_path + 'data\\processed\\ladder_teams.xlsx', index=False)
logger.info("A base foi exportada como csv e planilha do Excel!")

2022-09-29 00:55:42,618 - INFO - A base foi exportada como csv e planilha do Excel!
